# Project - YouTube Videos Summary Generator

### OpenAI Authentication

In [ ]:
import openai
import os

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')

### import custom package and setup path for it

In [ ]:
import sys
from pathlib import Path

# in jupyter (lab / notebook), based on notebook path

# print(f"Path.cwd(): {Path.cwd()}")
module_path = str(Path.cwd().parents[0])

if module_path not in sys.path:
    sys.path.append(module_path)

from common.usage import print_token_usage

In [ ]:
# !pip -q install pytube

In [ ]:
import os
from pytube import YouTube
